In [1]:
!pip install -q py7zr

In [1]:
import py7zr
import shutil
from pathlib import Path

In [11]:
root = Path('./data')
input_path = Path('./input/cifar-10')

In [10]:
# done
# with py7zr.SevenZipFile(input_path/'train.7z', mode='r') as z:
#     z.extractall(root)

# with py7zr.SevenZipFile(input_path/'test.7z', mode='r') as z:
#     z.extractall(root)

# shutil.copy(input_path/'trainLabels.csv', root/'trainLabels.csv')

# (root/'train').rename(root/'original_train')
# (root/'test').rename(root/'original_test')

WindowsPath('data/original_test')

In [2]:
import pandas as pd 
labels = pd.read_csv(root/'trainLabels.csv')
labels.head()

NameError: name 'root' is not defined

In [12]:
def copy_file(source , dest , file):
    dest.mkdir(parents = True , exist_ok = True)
    shutil.copy(source/file , dest/file)

In [34]:
import os
import random
def make_data_train_valid(root , labels , valid_vatio):
    source = Path(root/'original_train')
    with os.scandir(source) as it:
        for entry in it:
            if entry.is_file():
                img_id = entry.name.split('.')[0]
                img_class = labels[labels.id == int(img_id)].values[0][1]
                # img_class = labels[labels.id == int(img_id)].label.values[0]
                img_dir = Path('train') if random.random() > valid_vatio else Path('valid')
                copy_file(source , root/img_dir/img_class , entry.name)
                copy_file(source , root/'train_valid'/img_class , entry.name)

def make_data_test(root):
    source = Path(root/'original_test')
    with os.scandir(source) as it:
        for entry in it:
            if entry.is_file():
                img_dir = Path('test')
                copy_file(source , root/img_dir/'undefined' , entry.name)

In [32]:
# make_data_train_valid(root , labels , 0.1) # done

KeyboardInterrupt: 

In [35]:
# done
# make_data_test(root)
# 上面的操作把数据分成4个folder:train valid tran_valid test

In [13]:
import torch
import torchvision
from torchvision import transforms as ttf

In [14]:
from torch.utils.data import DataLoader

In [15]:
train_data = torchvision.datasets.ImageFolder(
    root/'train' , 
    transform = ttf.Compose([
        ttf.Resize((224 , 224)) , 
        ttf.ToTensor() , 
    ])
)
train_iter = DataLoader(train_data , batch_size = 128 , shuffle = False)

In [20]:
means = []
std = []
for X , Y in train_iter:
    means.append(X.mean(dim = (0 , 2 , 3))) # 0-batch_size 1-channel(RGB) 2-height 3 = weight
    std.append(X.std(dim = (0 , 2 , 3)))

In [22]:
!pip install tqdm
torch.stack(means , dim = 0).shape

torch.Size([353, 3])

In [47]:
from tqdm import tqdm
torch.stack(means , dim = 0).mean(dim = 0).shape

torch.Size([3])

In [24]:
meanv = torch.stack(means , dim = 0).mean(dim = 0) # 对三个通道求总的平均值
stdv = torch.stack(std,  dim = 0).mean(dim = 0)

In [26]:
train_ttf = ttf.Compose([
    ttf.Resize((224 , 224)) , 
    ttf.AutoAugment(policy=ttf.AutoAugmentPolicy.CIFAR10) , 
    ttf.ToTensor() , 
    ttf.Normalize(meanv , stdv) , 
])

valid_ttf = ttf.Compose([
    ttf.Resize((224 , 224)) , 
    ttf.ToTensor() , 
    ttf.Normalize(meanv , stdv) ,
])

In [28]:
train_data , train_valid_data = [
    torchvision.datasets.ImageFolder(fold , transform = train_ttf) for fold in [root/'train' , root/'train_valid']
]

valid_data , test_data = [
    torchvision.datasets.ImageFolder(fold , transform = valid_ttf) for fold in [root/'valid' , root/'test']
]

In [31]:
train_iter = DataLoader(train_data , batch_size = 128 , shuffle = True , num_workers = 2)
train_valid_iter = DataLoader(train_valid_data , batch_size = 128 , shuffle = True , num_workers = 2)
valid_iter = DataLoader(valid_data , batch_size = 256 , shuffle = False , num_workers = 2)
test_iter = DataLoader(test_data , batch_size = 256 , shuffle = False , num_workers = 2)

In [35]:
import torch.nn as nn
import torch.nn.functional as F

In [52]:
def get_net():
    resnet = torchvision.models.resnet34(pretrained = True)
    resnet.fc = nn.Linear(resnet.fc.in_features , 10)
    nn.init.xavier_uniform_(resnet.fc.weight)
    return resnet

In [53]:
import time
def train(net, train_dataloader, valid_dataloader, criterion, optimizer, epochs=10, device='cpu'):
    start = time.time()
    for epoch in range(1 , epochs + 1):
        print(f'epoch {epoch} :' )
        net.train()
        train_loss = 0.0
        train_acc = 0.0
        for X , y in tqdm(train_dataloader):
            X = X.to(device)
            y = y.to(device)
            pred = net(X)
            loss = criterion(pred , y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                train_loss += loss * len(X)
                train_acc += (torch.argmax(pred , dim = 1) == y).sum()
                
        if valid_dataloder is not None:
            net.eval()
            valid_loss = 0.0
            valid_acc = 0.0
            with torch.no_grad():
                for X ,y in tqdm(valid_dataloader):
                    X.to(device)
                    y.to(device)
                    pred = net(X)
                    loss = criterion(pred , y)
                    
                    valid_loss += loss * len(X)
                    valid_acc += (torch.argmax(pred , dim = 1) == y).sum()
        
        print(f'train_loss : {train_loss / len(train_dataloader.dataset)} , train_acc : {train_acc / len(train_dataloader.dataset)}')
        if valid_dataloader is not None:
            print(f'valid_loss : {valid_loss / len(train_dataloader.dataset)} , valid_acc : {valid_acc / len(train_dataloader.dataset)}')
        
    end = time.time()
    print(f'Total training time : {time} seconds')
    
    return net

In [56]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr , weight_decay , epochs = 1e-5 , 5e-4 , 20
net = get_net().to(device)
criterion = nn.CrossEntropyLoss()
params_1x = [param for name, param in net.named_parameters() if 'fc' not in str(name)]
optimizer = torch.optim.Adam([{'params':params_1x}, {'params': net.fc.parameters(), 'lr': lr*10}], lr=lr, weight_decay=weight_decay)

net = train(net, train_iter, valid_iter, criterion, optimizer, epochs, device)

epoch 1 :


 15%|███████████▋                                                                   | 52/353 [12:09<1:10:22, 14.03s/it]


KeyboardInterrupt: 

500.0
387.13184134056354
299.7421251594705
232.07944168063895
179.69068319023137
139.12797011035622
107.7217345015942
83.40502686000293
64.57748325074421
50.0
